In [396]:
import handcalcs.render
import numpy as np
import pandas as pd
from math import sqrt, atan, radians, sin, cos

# Reinforced concrete beam design (as per ACI 318-19)

## Loads

In [397]:
Nu = 0 # axil loads positive for compression, negative for tension

## Material properties

In [398]:
%%render

f_prime_c = 4000 # psi, unconfined concrete compressive stress
epsilon_cmax = 0.003 # max strain at the extreme compression fiber, from ACI 318
alpha_1 = 0.85

f_yl = 40000 # ksi, longitudinal rebar yield strength
epsilon_y = 0.002 # strain at longitudinal rebar yielding, from ACI 318

f_yt = 40000 # ksi, transverse rebar yield strength
phi_v = 0.75

<IPython.core.display.Latex object>

## Section details

In [399]:
%%render
b = 12 # inch, width of beam
h = 19 # inch, overall depth of beam
n_tensionbar = 7 # number of rebar on tension side
n_leg=2 # number of shear reinforcement legs per s distance
db_l = 8 # longitudinal bar size, US rebar size
db_t = 4 # transverse bar size, US rebar size
s = 6 # inch, transverse bar spacing
cover = 2 # inch, clear cover

<IPython.core.display.Latex object>

In [400]:
USrebar = pd.DataFrame({
    'size #': [ 4,    5,     6,   7  ,  8 ,    9,   10,   11],
    'area': [  0.2, 0.31, 0.44,  0.6, 0.79,    1,   1.27, 1.56]
})
As=float(USrebar.loc[USrebar['size #'] == db_l, 'area'])*n_tension # total tension rebar area

## Flexural resistance calculation

In [401]:
%%render long
Ag = b*h # inch^2, gross section area
d = h - cover - db_t/8-db_l/8/2 # inch, distance between compression edge and center of tension rebar
rho = As/(b*d) # longitudinal reinforcement ratio 

<IPython.core.display.Latex object>

In [402]:
%%render 2
if f_prime_c < 4000: beta_1 = 0.85
elif f_prime_c < 8000: beta_1 = 0.85-0.05*(f_prime_c-4000)/1000
else: beta_1 = 0.65

<IPython.core.display.Latex object>

In [403]:
%%render 3
a = (f_yl*As)/(alpha_1*f_prime_c*b) # inch, rectangular compression area depth
c = a / beta_1 # inch, compression edge to neutral axis
epsilon_t = epsilon_cmax*(d-c)/c # tensile strain in rebar

<IPython.core.display.Latex object>

In [404]:
%%render 3
if epsilon_t > (epsilon_y+0.003): phi_b=0.9
elif epsilon_t > epsilon_y: phi_b=0.65+0.25*(epsilon_t-epsilon_y)/0.003
else: phi_b = 0.65

<IPython.core.display.Latex object>

In [405]:
%%render 1
M_n = phi_b*As*f_yl*(d-a/2)/12/1000 # kip-ft

<IPython.core.display.Latex object>

## One way shear resistance calculation


In [406]:
Av = float(USrebar.loc[USrebar['size #'] == db_t, 'area'])*n_leg # in^2, shear reinforcement area within spacing s

In [407]:
%%render 
d = max(d, 0.8*h) # extreme compression firber to rebar centroid
rho_w = As/(b*d) # ratio of As to bd
lamb_s = min(sqrt(2/(1+d/10)),1) # reduction factor for size effect

<IPython.core.display.Latex object>

In [408]:
%%render 
Av_min = max(0.75*sqrt(f_prime_c)*b*s/(f_yt),50*b*s/(f_yt)) # min shear reinforcement for column

<IPython.core.display.Latex object>

In [409]:
%%render 1
Vc_a = (2*sqrt(f_prime_c)+Nu/6/Ag)*b*d/1000 #kips
Vc_b = (8*rho_w**(1/3)*sqrt(f_prime_c)+Nu/6/Ag)*b*d/1000 #kips
if Av>Av_min: Vc = max(Vc_a,Vc_b)/1000 #kips
else: Vc = (8*lamb_s*rho_w**(1/3)*sqrt(f_prime_c)+Nu/6/Ag)*b*d/1000 #kips

<IPython.core.display.Latex object>

In [410]:
%%render 1
Vs = (Av*f_yt*d)/s/1000 #kips
Vn = Vc +Vs #kips

<IPython.core.display.Latex object>

In [414]:
%%render 2
V_factored = Vn * phi_v #kips

<IPython.core.display.Latex object>